# Ensuring Consistency with ACID Transactions with Delta Lake (Loan Risk Data)

<img src="https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-logo-whitebackground.png" width=200/>

This is a companion notebook to provide a Delta Lake example against the Lending Club data.
* This notebook has been tested with *DBR 5.4 ML Beta, Python 3*
* For the bottom sections of this notebook, you will need `mlflow` and `yellowbrick` installed on your cluster as well

## The Data

The data used is public data from Lending Club. It includes all funded loans from 2012 to 2017. Each loan includes applicant information provided by the applicant as well as the current loan status (Current, Late, Fully Paid, etc.) and latest payment information. For a full view of the data please view the data dictionary available [here](https://resources.lendingclub.com/LCDataDictionary.xlsx).


![Loan_Data](https://preview.ibb.co/d3tQ4R/Screen_Shot_2018_02_02_at_11_21_51_PM.png)

https://www.kaggle.com/wendykan/lending-club-loan-data

## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Delta Lake

Optimization Layer a top blob storage for Reliability (i.e. ACID compliance) and Low Latency of Streaming + Batch data pipelines.

## Import Data and create pre-Delta Lake Table
* This will create a lot of small Parquet files emulating the typical small file problem that occurs with streaming or highly transactional data

In [5]:
# -----------------------------------------------
# Uncomment and run if this folder does not exist
# -----------------------------------------------
# Configure location of loanstats_2012_2017.parquet
lspq_path = "/databricks-datasets/samples/lending_club/parquet/"

# Read loanstats_2012_2017.parquet
data = spark.read.parquet(lspq_path)

# Reduce the amount of data (to run on DBCE)
(loan_stats, loan_stats_rest) = data.randomSplit([0.01, 0.99], seed=123)

# Select only the columns needed
loan_stats = loan_stats.select("addr_state", "loan_status")

# Create loan by state
loan_by_state = loan_stats.groupBy("addr_state").count()

# Create table
loan_by_state.createOrReplaceTempView("loan_by_state")

# Display loans by state
display(loan_by_state)

addr_state,count
AZ,329
SC,174
LA,167
MN,256
NJ,541
DC,38
OR,178
VA,413
RI,66
WY,31


## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Easily Convert Parquet to Delta Lake format
With Delta Lake, you can easily transform your Parquet data into Delta Lake format.

In [7]:
# Configure Delta Lake Silver Path
DELTALAKE_SILVER_PATH = "/ml/loan_by_state_delta"

# Remove folder if it exists
dbutils.fs.rm(DELTALAKE_SILVER_PATH, recurse=True)

Out[2]: True

In [8]:
%sql 
-- Current example is creating a new table instead of in-place import so will need to change this code
DROP TABLE IF EXISTS loan_by_state_delta;

CREATE TABLE loan_by_state_delta
USING delta
LOCATION '/ml/loan_by_state_delta'
AS SELECT * FROM loan_by_state;

-- View Delta Lake table
SELECT * FROM loan_by_state_delta

addr_state,count
CA,2016
MS,84
GA,495
MA,311
SD,31
NV,219
OH,504
MT,28
ID,15
NH,68


In [9]:
%sql 
DESCRIBE DETAIL delta.`/ml/loan_by_state_delta`

format,id,name,description,location,createdAt,lastModified,partitionColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion
delta,c23bbd08-ce4c-4818-b5d3-34ef44f866b0,null,null,dbfs:/ml/loan_by_state_delta,2020-02-11T18:37:31.177+0000,2020-02-11T18:38:46.000+0000,List(),46,30502,Map(),1,2


## Stop the notebook before the streaming cell, in case of a "run all"

In [11]:
dbutils.notebook.exit("stop") 

stop

In [12]:
%fs ls /ml/loan_by_state_delta/_delta_log/

path,name,size
dbfs:/ml/loan_by_state_delta/_delta_log/.s3-optimization-0,.s3-optimization-0,0
dbfs:/ml/loan_by_state_delta/_delta_log/.s3-optimization-1,.s3-optimization-1,0
dbfs:/ml/loan_by_state_delta/_delta_log/.s3-optimization-2,.s3-optimization-2,0
dbfs:/ml/loan_by_state_delta/_delta_log/00000000000000000000.crc,00000000000000000000.crc,91
dbfs:/ml/loan_by_state_delta/_delta_log/00000000000000000000.json,00000000000000000000.json,16690


## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Unified Batch and Streaming Source and Sink

These cells showcase streaming and batch concurrent queries (inserts and reads)
* This notebook will run an `INSERT` every 10s against our `loan_stats_delta` table
* We will run two streaming queries concurrently against this data
* Note, you can also use `writeStream` but this version is easier to run in DBCE

In [14]:
# Read the insertion of data
loan_by_state_readStream = spark.readStream.format("delta").load(DELTALAKE_SILVER_PATH)
loan_by_state_readStream.createOrReplaceTempView("loan_by_state_readStream")

In [15]:
%sql
select addr_state, sum(`count`) as loans from loan_by_state_readStream group by addr_state

addr_state,loans
AZ,329
SC,174
LA,167
MN,256
NJ,541
DC,38
OR,178
VA,413
RI,66
WY,31


**Wait** until the stream is up and running before executing the code below

In [17]:
import time
i = 1
while i <= 6:
  # Execute Insert statement
  insert_sql = "INSERT INTO loan_by_state_delta VALUES ('IA', 450)"
  spark.sql(insert_sql)
  print('loan_by_state_delta: inserted new row of data, loop: [%s]' % i)
    
  # Loop through
  i = i + 1
  time.sleep(5)

In [18]:
%fs ls /ml/loan_by_state_delta/_delta_log/

path,name,size
dbfs:/ml/loan_by_state_delta/_delta_log/.s3-optimization-0,.s3-optimization-0,0
dbfs:/ml/loan_by_state_delta/_delta_log/.s3-optimization-1,.s3-optimization-1,0
dbfs:/ml/loan_by_state_delta/_delta_log/.s3-optimization-2,.s3-optimization-2,0
dbfs:/ml/loan_by_state_delta/_delta_log/00000000000000000000.crc,00000000000000000000.crc,91
dbfs:/ml/loan_by_state_delta/_delta_log/00000000000000000000.json,00000000000000000000.json,16690
dbfs:/ml/loan_by_state_delta/_delta_log/00000000000000000001.crc,00000000000000000001.crc,91
dbfs:/ml/loan_by_state_delta/_delta_log/00000000000000000001.json,00000000000000000001.json,677
dbfs:/ml/loan_by_state_delta/_delta_log/00000000000000000002.crc,00000000000000000002.crc,91
dbfs:/ml/loan_by_state_delta/_delta_log/00000000000000000002.json,00000000000000000002.json,677
dbfs:/ml/loan_by_state_delta/_delta_log/00000000000000000003.crc,00000000000000000003.crc,91


**Note**: Once the previous cell is finished and the state of Iowa is fully populated in the map (in cell 14), click *Cancel* in Cell 14 to stop the `readStream`.

Let's review our current set of loans using our map visualization.

In [21]:
%sql
-- Review current loans within the `loan_by_state_delta` Delta Lake table
select addr_state, sum(`count`) as loans from loan_by_state_delta group by addr_state

addr_state,loans
AZ,329
SC,174
LA,167
MN,256
NJ,541
DC,38
OR,178
VA,413
RI,66
WY,31


Observe that the Iowa (middle state) has the largest number of loans due to the recent stream of data.  Note that the original `loan_by_state_delta` table is updated as we're reading `loan_by_state_readStream`.

##![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Full DML Support

**Note**: Full DML Support is a feature that will be coming soon to Delta Lake; the preview is currently available in Databricks.

Delta Lake supports standard DML including UPDATE, DELETE and MERGE INTO providing developers more controls to manage their big datasets.

Let's start by creating a traditional Parquet table

In [26]:
# Load new DataFrame based on current Delta table
lbs_df = sql("select * from loan_by_state_delta")

# Save DataFrame to Parquet
lbs_df.write.mode("overwrite").parquet("loan_by_state.parquet")

# Reload Parquet Data
lbs_pq = spark.read.parquet("loan_by_state.parquet")

# Create new table on this parquet data
lbs_pq.createOrReplaceTempView("loan_by_state_pq")

# Review data
display(sql("select * from loan_by_state_pq"))

addr_state,count
IA,450
IA,450
ME,15
LA,167
MO,223
RI,66
IL,578
OH,504
MT,28
IA,450


###![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) DELETE Support

The data was originally supposed to be assigned to `WA` state, so let's `DELETE` those values assigned to `IA`

In [28]:
%sql
-- Attempting to run `DELETE` on the Parquet table
DELETE FROM loan_by_state_pq WHERE addr_state = 'IA'

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: DELETE destination only supports Delta sources.
Some(Relation[addr_state#8628,count#8629L] parquet
);
	at com.databricks.sql.transaction.tahoe.DeltaErrors$.notADeltaSourceException(DeltaErrors.scala:166)
	at com.databricks.sql.transaction.tahoe.PreprocessTableDelete$$anonfun$apply0$1.applyOrElse(PreprocessTableDelete.scala:50)
	at com.databricks.sql.transaction.tahoe.PreprocessTableDelete$$anonfun$apply0$1.applyOrElse(PreprocessTableDelete.scala:44)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsDown$1$$anonfun$2.apply(AnalysisHelper.scala:108)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsDown$1$$anonfun$2.apply(AnalysisHelper.scala:108)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsDown$1.apply(AnalysisHelper.scala:107)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsDown$1.apply(AnalysisHelper.scala:106)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:194)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$class.resolveOperatorsDown(AnalysisHelper.scala:106)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsDown(LogicalPlan.scala:29)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$class.resolveOperators(AnalysisHelper.scala:73)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperators(LogicalPlan.scala:29)
	at com.databricks.sql.transaction.tahoe.PreprocessTableDelete.apply0(PreprocessTableDelete.scala:44)
	at com.databricks.sql.transaction.tahoe.PreprocessTableDelete.apply(PreprocessTableDelete.scala:40)
	at com.databricks.sql.transaction.tahoe.PreprocessTableDelete.apply(PreprocessTableDelete.scala:34)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1$$anonfun$apply$1.apply(RuleExecutor.scala:112)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1$$anonfun$apply$1.apply(RuleExecutor.scala:109)
	at scala.collection.LinearSeqOptimized$class.foldLeft(LinearSeqOptimized.scala:124)
	at scala.collection.immutable.List.foldLeft(List.scala:84)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1.apply(RuleExecutor.scala:109)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1.apply(RuleExecutor.scala:101)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.execute(RuleExecutor.scala:101)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.org$apache$spark$sql$catalyst$analysis$Analyzer$$executeSameContext(Analyzer.scala:137)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.execute(Analyzer.scala:131)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.execute(Analyzer.scala:103)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$executeAndTrack$1.apply(RuleExecutor.scala:80)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$executeAndTrack$1.apply(RuleExecutor.scala:80)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker$.withTracker(QueryPlanningTracker.scala:88)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.executeAndTrack(RuleExecutor.scala:79)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$$anonfun$executeAndCheck$1.apply(Analyzer.scala:115)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$$anonfun$executeAndCheck$1.apply(Analyzer.scala:114)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.markInAnalyzer(AnalysisHelper.scala:201)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.executeAndCheck(Analyzer.scala:114)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$analyzed$1.apply(QueryExecution.scala:85)
	at org.apa

**Note**: This command fails because the `DELETE` statements are not supported in Parquet, but are supported in Delta Lake.

In [30]:
%sql
-- Running `DELETE` on the Delta Lake table
DELETE FROM loan_by_state_delta WHERE addr_state = 'IA'

In [31]:
%sql
-- Review current loans within the `loan_by_state_delta` Delta Lake table
select addr_state, sum(`count`) as loans from loan_by_state_delta group by addr_state

addr_state,loans
AZ,329
SC,174
LA,167
MN,256
NJ,541
DC,38
OR,178
VA,413
RI,66
WY,31


###![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) UPDATE Support
The data was originally supposed to be assigned to `WA` state, so let's `UPDATE` those values

In [33]:
%sql
-- Attempting to run `UPDATE` on the Parquet table
UPDATE loan_by_state_pq SET `count` = 2700 WHERE addr_state = 'WA'

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: UPDATE destination only supports Delta sources.
Some(Relation[addr_state#8628,count#8629L] parquet
);
	at com.databricks.sql.transaction.tahoe.DeltaErrors$.notADeltaSourceException(DeltaErrors.scala:166)
	at com.databricks.sql.transaction.tahoe.PreprocessTableUpdateEdge$$anonfun$apply0$1.applyOrElse(PreprocessTableUpdateEdge.scala:56)
	at com.databricks.sql.transaction.tahoe.PreprocessTableUpdateEdge$$anonfun$apply0$1.applyOrElse(PreprocessTableUpdateEdge.scala:50)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsDown$1$$anonfun$2.apply(AnalysisHelper.scala:108)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsDown$1$$anonfun$2.apply(AnalysisHelper.scala:108)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsDown$1.apply(AnalysisHelper.scala:107)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsDown$1.apply(AnalysisHelper.scala:106)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:194)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$class.resolveOperatorsDown(AnalysisHelper.scala:106)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsDown(LogicalPlan.scala:29)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$class.resolveOperators(AnalysisHelper.scala:73)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperators(LogicalPlan.scala:29)
	at com.databricks.sql.transaction.tahoe.PreprocessTableUpdateEdge.apply0(PreprocessTableUpdateEdge.scala:50)
	at com.databricks.sql.transaction.tahoe.PreprocessTableUpdateEdge.apply(PreprocessTableUpdateEdge.scala:46)
	at com.databricks.sql.transaction.tahoe.PreprocessTableUpdateEdge.apply(PreprocessTableUpdateEdge.scala:39)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1$$anonfun$apply$1.apply(RuleExecutor.scala:112)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1$$anonfun$apply$1.apply(RuleExecutor.scala:109)
	at scala.collection.LinearSeqOptimized$class.foldLeft(LinearSeqOptimized.scala:124)
	at scala.collection.immutable.List.foldLeft(List.scala:84)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1.apply(RuleExecutor.scala:109)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1.apply(RuleExecutor.scala:101)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.execute(RuleExecutor.scala:101)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.org$apache$spark$sql$catalyst$analysis$Analyzer$$executeSameContext(Analyzer.scala:137)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.execute(Analyzer.scala:131)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.execute(Analyzer.scala:103)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$executeAndTrack$1.apply(RuleExecutor.scala:80)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$executeAndTrack$1.apply(RuleExecutor.scala:80)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker$.withTracker(QueryPlanningTracker.scala:88)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.executeAndTrack(RuleExecutor.scala:79)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$$anonfun$executeAndCheck$1.apply(Analyzer.scala:115)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$$anonfun$executeAndCheck$1.apply(Analyzer.scala:114)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.markInAnalyzer(AnalysisHelper.scala:201)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.executeAndCheck(Analyzer.scala:114)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$analyzed$1.ap

**Note**: This command fails because the `UPDATE` statements are not supported in Parquet, but are supported in Delta Lake.

In [35]:
%sql
-- Running `UPDATE` on the Delta Lake table
UPDATE loan_by_state_delta SET `count` = 2700 WHERE addr_state = 'WA'

In [36]:
%sql
-- Review current loans within the `loan_by_state_delta` Delta Lake table
select addr_state, sum(`count`) as loans from loan_by_state_delta group by addr_state

addr_state,loans
AZ,329
SC,174
LA,167
MN,256
NJ,541
DC,38
OR,178
VA,413
RI,66
WY,31


###![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) MERGE INTO Support

#### INSERT or UPDATE parquet: 7-step process

With a legacy data pipeline, to insert or update a table, you must:
1. Identify the new rows to be inserted
2. Identify the rows that will be replaced (i.e. updated)
3. Identify all of the rows that are not impacted by the insert or update
4. Create a new temp based on all three insert statements
5. Delete the original table (and all of those associated files)
6. "Rename" the temp table back to the original table name
7. Drop the temp table

![](https://pages.databricks.com/rs/094-YMS-629/images/merge-into-legacy.gif)


#### INSERT or UPDATE with Delta Lake

2-step process: 
1. Identify rows to insert or update
2. Use `MERGE`

In [38]:
# Let's create a simple table to merge
items = [('IA', 10), ('CA', 2500), ('OR', None)]
cols = ['addr_state', 'count']
merge_table = spark.createDataFrame(items, cols)
merge_table.createOrReplaceTempView("merge_table")
display(merge_table)

addr_state,count
IA,10
CA,2500
OR,null


Instead of writing separate `INSERT` and `UPDATE` statements, we can use a `MERGE` statement.

In [40]:
%sql
MERGE INTO loan_by_state_delta as d
USING merge_table as m
on d.addr_state = m.addr_state
WHEN MATCHED THEN 
  UPDATE SET *
WHEN NOT MATCHED 
  THEN INSERT *

In [41]:
%sql
-- Review current loans within the `loan_by_state_delta` Delta Lake table
select addr_state, sum(`count`) as loans from loan_by_state_delta group by addr_state

addr_state,loans
AZ,329
SC,174
LA,167
MN,256
NJ,541
DC,38
OR,null
VA,413
RI,66
WY,31


##![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Schema Evolution
With the `mergeSchema` option, you can evolve your Delta Lake table schema

In [43]:
# Generate new loans with dollar amounts 
loans = sql("select addr_state, cast(rand(10)*count as bigint) as count, cast(rand(10) * 10000 * count as double) as amount from loan_by_state_delta")
display(loans)

addr_state,count,amount
MS,34,347518.62365619664
GA,361,3619301.04453882
MA,280,2808859.0592417694
SD,2,29233.63585172156
NV,83,839657.0655492826
OH,280,2806900.0523038497
MT,13,139368.359385189
ID,2,29743.794573125953
NH,8,86456.43192777676
AL,139,1391590.222073422


In [44]:
# Let's write this data out to our Delta table
loans.write.format("delta").mode("append").save(DELTALAKE_SILVER_PATH)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o1087.save.
: org.apache.spark.sql.AnalysisException: A schema mismatch detected when writing to the Delta table.
To enable schema migration, please set:
'.option("mergeSchema", "true")'.

Table schema:
root
-- addr_state: string (nullable = true)
-- count: long (nullable = true)


Data schema:
root
-- addr_state: string (nullable = true)
-- count: long (nullable = true)
-- amount: double (nullable = true)

 
If Table ACLs are enabled, these options will be ignored. Please use the ALTER TABLE
command for changing the schema.
 ;
	at com.databricks.sql.transaction.tahoe.MetadataMismatchErrorBuilder.finalizeAndThrow(DeltaErrors.scala:840)
	at com.databricks.sql.transaction.tahoe.schema.ImplicitMetadataOperation$class.updateMetadata(ImplicitMetadataOperation.scala:125)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDelta.updateMetadata(WriteIntoDelta.scala:50)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDelta.write(WriteIntoDelta.scala:91)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDelta$$anonfun$run$1$$anonfun$apply$1.apply(WriteIntoDelta.scala:71)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDelta$$anonfun$run$1$$anonfun$apply$1.apply(WriteIntoDelta.scala:70)
	at com.databricks.sql.transaction.tahoe.DeltaLog.withNewTransaction(DeltaLog.scala:409)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDelta$$anonfun$run$1.apply(WriteIntoDelta.scala:70)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDelta$$anonfun$run$1.apply(WriteIntoDelta.scala:69)
	at com.databricks.sql.acl.CheckPermissions$.trusted(CheckPermissions.scala:812)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDelta.run(WriteIntoDelta.scala:69)
	at com.databricks.sql.transaction.tahoe.sources.DeltaDataSource.createRelation(DeltaDataSource.scala:128)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:45)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:70)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:68)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.doExecute(commands.scala:86)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:146)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:134)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$5.apply(SparkPlan.scala:187)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:183)
	at org.apache.spark.sql.execution.SparkPlan.execute(SparkPlan.scala:134)
	at org.apache.spark.sql.execution.QueryExecution.toRdd$lzycompute(QueryExecution.scala:115)
	at org.apache.spark.sql.execution.QueryExecution.toRdd(QueryExecution.scala:115)
	at org.apache.spark.sql.DataFrameWriter$$anonfun$runCommand$1.apply(DataFrameWriter.scala:710)
	at org.apache.spark.sql.DataFrameWriter$$anonfun$runCommand$1.apply(DataFrameWriter.scala:710)
	at org.apache.spark.sql.execution.SQLExecution$$anonfun$withCustomExecutionEnv$1.apply(SQLExecution.scala:111)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.scala:240)
	at org.apache.spark.sql.execution.SQLExecution$.withCustomExecutionEnv(SQLExecution.sc

**Note**: This command fails because the schema of our new data does not match the schema of our original data

In [46]:
# Add the mergeSchema option
loans.write.option("mergeSchema","true").format("delta").mode("append").save(DELTALAKE_SILVER_PATH)

**Note**: With the `mergeSchema` option, we can merge these different schemas together.

In [48]:
%sql
-- Review current loans within the `loan_by_state_delta` Delta Lake table
select addr_state, sum(`amount`) as amount from loan_by_state_delta group by addr_state order by sum(`amount`) desc limit 10

addr_state,amount
CA,2.182365626907857E7
FL,8690354.473922366
IL,5033186.70166422
NJ,4143498.223235212
VA,3804654.67458381
GA,3619301.04453882
MD,3166168.859533876
AZ,2980741.078421011
WA,2922134.708269761
MA,2808859.0592417694


## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Let's Travel back in Time!
Databricks Delta’s time travel capabilities simplify building data pipelines for the following use cases. 

* Audit Data Changes
* Reproduce experiments & reports
* Rollbacks

As you write into a Delta table or directory, every operation is automatically versioned.

You can query by:
1. Using a timestamp
1. Using a version number

using Python, Scala, and/or Scala syntax; for these examples we will use the SQL syntax.  

For more information, refer to [Introducing Delta Time Travel for Large Scale Data Lakes](https://databricks.com/blog/2019/02/04/introducing-delta-time-travel-for-large-scale-data-lakes.html)

### ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Review Delta Lake Table History
All the transactions for this table are stored within this table including the initial set of insertions, update, delete, merge, and inserts with schema modification

In [51]:
%sql
DESCRIBE HISTORY loan_by_state_delta

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend
10,2020-02-11T19:25:41.000+0000,302794,denny.g.lee@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(1421132891622955),0211-160410-blabs369,9,WriteSerializable,false
9,2020-02-11T19:19:00.000+0000,302794,denny.g.lee@gmail.com,MERGE,Map(predicate -> (d.`addr_state` = m.`addr_state`)),null,List(1421132891622955),0211-160410-blabs369,8,WriteSerializable,false
8,2020-02-11T19:14:07.000+0000,302794,denny.g.lee@gmail.com,UPDATE,Map(predicate -> (addr_state#9240 = WA)),null,List(1421132891622955),0211-160410-blabs369,7,WriteSerializable,false
7,2020-02-11T19:01:17.000+0000,302794,denny.g.lee@gmail.com,DELETE,"Map(predicate -> [""(default.loan_by_state_delta.`addr_state` = 'IA')""])",null,List(1421132891622955),0211-160410-blabs369,6,WriteSerializable,false
6,2020-02-11T18:51:49.000+0000,302794,denny.g.lee@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(1421132891622955),0211-160410-blabs369,5,WriteSerializable,true
5,2020-02-11T18:50:37.000+0000,302794,denny.g.lee@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(1421132891622955),0211-160410-blabs369,4,WriteSerializable,true
4,2020-02-11T18:49:37.000+0000,302794,denny.g.lee@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(1421132891622955),0211-160410-blabs369,3,WriteSerializable,true
3,2020-02-11T18:48:43.000+0000,302794,denny.g.lee@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(1421132891622955),0211-160410-blabs369,2,WriteSerializable,true
2,2020-02-11T18:47:53.000+0000,302794,denny.g.lee@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(1421132891622955),0211-160410-blabs369,1,WriteSerializable,true
1,2020-02-11T18:47:29.000+0000,302794,denny.g.lee@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(1421132891622955),0211-160410-blabs369,0,WriteSerializable,true


### ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Time Travel via Version Number
Below are SQL syntax examples of Delta Time Travel by using a Version Number

In [53]:
%sql
SELECT * FROM loan_by_state_delta VERSION AS OF 0

addr_state,count
CA,2016
MS,84
GA,495
MA,311
SD,31
NV,219
OH,504
MT,28
ID,15
NH,68


In [54]:
%sql
SELECT * FROM loan_by_state_delta VERSION AS OF 9

addr_state,count
MS,84
GA,495
MA,311
SD,31
NV,219
OH,504
MT,28
ID,15
NH,68
AL,183


## Run Our Model
Let's run a simple linear regression model to predict the number of loans based on the population of the state
* The following shell statements downloads the us_census_2020 data that we will join with the `loan_by_state_delta` table

In [56]:
%sh mkdir -p /dbfs/tmp/sais_eu_19_demo/census/ && wget -O /dbfs/tmp/sais_eu_19_demo/census/us_census_2010.csv https://pages.databricks.com/rs/094-YMS-629/images/us_census_2010.csv && ls -al /dbfs/tmp/sais_eu_19_demo/census/

--2020-02-11 19:44:27-- https://pages.databricks.com/rs/094-YMS-629/images/us_census_2010.csv
Resolving pages.databricks.com (pages.databricks.com)... 104.17.73.206, 104.17.74.206, 104.17.70.206, ...
Connecting to pages.databricks.com (pages.databricks.com)|104.17.73.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 591 [text/plain]
Saving to: ‘/dbfs/tmp/sais_eu_19_demo/census/us_census_2010.csv’

 0K 100% 69.5M=0s

2020-02-11 19:44:28 (69.5 MB/s) - ‘/dbfs/tmp/sais_eu_19_demo/census/us_census_2010.csv’ saved [591/591]

total 0
drwxr-xr-x 1 root root 0 Jan 1 1970 .
drwxr-xr-x 1 root root 0 Jan 1 1970 ..
-rw-r--r-- 1 root root 591 Feb 11 19:44 us_census_2010.csv

### Notes
If you forgot to install `mlflow` and `yellowbrick` on your cluster, instead of re-running everything all over again:
* Note that the Delta Lake table is stored in `DELTALAKE_SILVERPATH` or `/ml/loan_by_state_delta`
* You can add the libraries, restart the cluster and then start reading the data from the following cells (instead of rerunning everything all over again)
* Just uncomment the cell below to reconnect to your Delta Table

In [58]:
## Recreate loan_by_state_delta view
# spark.read.format("delta").load("/ml/loan_by_state_delta").createOrReplaceTempView("loan_by_state_delta")
## Check data
# display(spark.sql("select count(1) from loan_by_state_delta"))

In [59]:
# Include census data
census = spark.read.csv('/tmp/sais_eu_19_demo/census/us_census_2010.csv', sep=',', inferSchema=True, header=True)
census.createOrReplaceTempView("census")

# Data versions (0, 6, 9)
dfv0 = spark.sql("select c.Population, l.count as label from (select addr_state as State, count from loan_by_state_delta  version as of 0) l left outer join census c on c.State = l.State")
dfv6 = spark.sql("select c.Population, l.count as label from (select addr_state as State, count from loan_by_state_delta  version as of 6) l left outer join census c on c.State = l.State")
dfv9 = spark.sql("select c.Population, l.count as label from (select addr_state as State, count from loan_by_state_delta  version as of 9 where count is not null) l left outer join census c on c.State = l.State")

In [60]:
# Import MLflow
import mlflow
import mlflow.spark
#print("MLflow Version: %s" % mlflow.__version__)

# Display Residuals (yellowbrick)
def displayResiduals(train, test):
  from sklearn.linear_model import Ridge
  from yellowbrick.regressor import ResidualsPlot

  # define feature columns
  featureColumns = ['Population']

  # Create pandas DataFrames
  pdf_train = train.toPandas()
  pdf_test = test.toPandas()

  # Convert to X, y train and test values
  X_train = pdf_train[['Population']].to_numpy()
  y_train = pdf_train[['label']].to_numpy().flatten()
  X_test = pdf_test[['Population']].to_numpy()
  y_test = pdf_test[['label']].to_numpy().flatten()  
  
  # Instantiate the linear model and visualizer
  ridge = Ridge()
  visualizer = ResidualsPlot(ridge)

  # Visualize
  visualizer.fit(X_train, y_train)  # Fit the training data to the model
  visualizer.score(X_test, y_test)  # Evaluate the model on the test data
  visualizer.show(outpath="ridge-model-residuals.png")
  #fig=visualizer.poof()             # Draw/show/poof the data
  
# Predict Loan Count (based on state population)
def predictLoanCount(df, data_version):
  from pyspark.ml.linalg import Vectors
  from pyspark.ml.feature import VectorAssembler
  from pyspark.ml.regression import LinearRegression
  from pyspark.ml.evaluation import RegressionEvaluator
  
  # assemble vector
  assembler = VectorAssembler(
    inputCols=["Population"],
    outputCol="features")
  output = assembler.transform(df)

  # Log mlflow
  with mlflow.start_run() as run:  
    # Define LinearRegression algorithm
    lr = LinearRegression(maxIter=10, regParam=0.0, elasticNetParam=0.8)
    model = lr.fit(output)  

    # Calculate predictions
    predictions = model.transform(output)

    # calculate RMSE
    evaluator = RegressionEvaluator(metricName="rmse")
    RMSE = evaluator.evaluate(predictions)
    #print("Model: Root Mean Squared Error = " + str(RMSE))

    # Log Parameters
    mlflow.log_param("data version", data_version)
    mlflow.log_metric("RMSE", RMSE)

    # Log Model
    mlflow.spark.log_model(model, "model") 
    
    # Save if not data_version is "v0"
    if (data_version != 'v0'):
      # Log artifacts (output files)
      mlflow.log_artifact("ridge-model-residuals.png")
  
  # return predictions DataFrame
  #return predictions

In [61]:
# Calculate predictions
# Initial version of data (v0)
predictLoanCount(dfv0, 'v0')

# Version 6 (after streaming of Iowa data)
displayResiduals(dfv0, dfv6)
predictLoanCount(dfv6, 'v6')

# Version 9 (after correction of data: update, delete, merge)
displayResiduals(dfv0, dfv9)
predictLoanCount(dfv9, 'v9')

#### Review RMSE and Residuals
* Review the RMSE values via the MLflow Sidebar